In [228]:
import numpy as np

In [1]:
class LogisticRegression:
    def __init__(self,indims,outdims):
        
        self.indims=indims
        self.outdims=outdims
        self.weights=np.array([])
        
    def sigmoid(self,X):
        return 1/(1+np.exp(-X))

    def softmax(self,X):
        p=np.exp(X)
        return(p/(p.sum(1).reshape(X.shape[0],1)))
            
    def BCEC(self,y,p):
        Err=[]
        zeros=np.where(y==0)
        ones=np.where(y==1)
        Err=np.hstack((-np.log(p[ones]),-np.log(1-p[zeros])))
        return np.mean(Err)

    def GCEC(self,y,p):
        return -1*np.sum(np.multiply(y,np.log(p)))/y.shape[0]

    def weightinit(self):
        self.weights=np.random.randn(self.indims+1,self.outdims)
            
    def predict(self, X):
        if X.shape ==self.indims:
            X=np.hstack((np.ones((X.shape[0],1)),X))
        if not self.weights.size:
            self.weightinit()
        if self.outdims>1:
            self.probabilities=self.softmax(X@self.weights)
            self.predictions=np.eye(self.probabilities.shape[1])[np.argmax(self.probabilities, axis=1)]
        else:
            self.probabilities=self.sigmoid(X@self.weights)
            self.predictions=np.rint(self.probabilities)
            
            
    def train(self,X,y,epochs,eta,X_val=False,y_val=False,lam1=0,lam2=0):
        self.valerror=[]
        self.error=[]
        d={}
        X=np.hstack((np.ones((X.shape[0],1)),X))
        
        if (y.shape[1]<self.outdims):
            y=self.mat_ohe(y)
        

        self.predict(X)
 
        for i in range(epochs):
            self.weights=self.weights-eta*(X.T@(self.predictions-y))-lam1*self.weights-lam2*np.sign(self.weights)

            if X_val:
                self.predict(X_val)
                if outdims>1:
                    self.valerror.append(self.GCEC(y_val,self.probabilities))
                else:
                    self.valerror.append(self.BCEC(y_val,self.probabilities))
            self.predict(X)
            if self.outdims>1:
                self.error.append(self.GCEC(y,self.probabilities))
            else:
                self.error.append(self.BCEC(y,self.probabilities))
            